<img src="images/logo_city.png" align="right" width="20%">

# Training A Graph State Prediction Network (GSPNet): Simplified version

This notebook is a prototype on how to train a graph state prediction network using CNN and LSTM.
The content is devided into __3__ parts:

1. Data preprocessing
2. Model building, Training and Tuning
3. Prediction and per demand modifacation

The model is built with [PyTorch](https://pytorch.org/).

<p style='color: darkred'><strong>This version is simplified and most of intermediate explainatory codes and comments are removed except those directly contribute to generating images. For duck process, see the prototype version.</strong></p>

## Part 1: Data Preprocessing

### 1. Load Libraries and Data

Load in the data. Check the data integrity.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sqlalchemy
import time
import psycopg2
import warnings
import re
import torch
from PIL import Image
from IPython.display import display
from scipy import stats
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Load data:

In [2]:
table = pd.read_csv('dataset/nytaxi_yellow_2017_mar.csv')
table.head()

,tripid,tpep_pickup_datetime,tpep_dropoff_datetime,pulocationid,dolocationid,trip_distance,passenger_count,total_amount,trip_time,trip_avg_speed,trip_time_sec
0,35972297,2017-03-09 21:30:11,2017-03-09 21:44:20,148,48,4.06,1,18.36,00:14:09,17.215548,849
1,35972298,2017-03-09 21:47:00,2017-03-09 21:58:01,48,107,2.73,1,12.80,00:11:01,14.868381,661
2,35972299,2017-03-09 22:01:08,2017-03-09 22:11:16,79,162,2.27,1,14.12,00:10:08,13.440789,608
3,35972300,2017-03-09 22:16:05,2017-03-10 06:26:11,237,41,3.86,1,17.29,08:10:06,0.472557,29406
4,35972301,2017-03-31 06:31:53,2017-03-31 06:41:48,41,162,3.45,1,13.30,00:09:55,20.873950,595


Check table shape:

In [3]:
table.shape

(10222693, 11)

Check data:

Ratio of trips last longer than 30 minutes: (better smaller than 0.1)

In [5]:
(table.loc[table['trip_time_sec'] > 1800].shape[0] / table.shape[0])

0.08854594381343546

Project only needed columns:

In [6]:
# draw four needed columns and a id column, then sort according to time.
# the `tripid` column is for the sake of naming.
tensor_gen = table.loc[:,['tripid',
                          'tpep_pickup_datetime',
                          'tpep_dropoff_datetime',
                          'pulocationid', 'dolocationid']
                      ] # the first sort condition rules
tensor_gen.head()

,tripid,tpep_pickup_datetime,tpep_dropoff_datetime,pulocationid,dolocationid
0,35972297,2017-03-09 21:30:11,2017-03-09 21:44:20,148,48
1,35972298,2017-03-09 21:47:00,2017-03-09 21:58:01,48,107
2,35972299,2017-03-09 22:01:08,2017-03-09 22:11:16,79,162
3,35972300,2017-03-09 22:16:05,2017-03-10 06:26:11,237,41
4,35972301,2017-03-31 06:31:53,2017-03-31 06:41:48,41,162


### 2. Time Interval Preprocessing

Create time interval and process original table to generate images.

Change datetime columns from str type to Timestamp:

In [7]:
# Preprocess datatime columns here
# Very expensive operation, do once.
tensor_gen['tpep_pickup_datetime'] = pd.to_datetime(tensor_gen['tpep_pickup_datetime'])
tensor_gen['tpep_dropoff_datetime'] = pd.to_datetime(tensor_gen['tpep_dropoff_datetime'])

tensor_gen.dtypes

tripid                            int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
pulocationid                      int64
dolocationid                      int64
dtype: object

Function to create time **intervals**:

In [9]:
def timesplit(stp: str, etp: str, freq='10min'):
    '''
    Create a DatetimeIndx interval.
    
    Params:
        stp: string, starting time point, first left bound
        etp: string, ending time point, last right bound
        freq: frequency, time interval unit of the splice operation
    The stp and etp must of pattern "yyyy-mm-dd hh:mm:ss", otherwise exception will be raised.
    
    Return:
        A list of time intervals tuples,each item is a tuple of two
        interval(i.e., pandas.core.indexes.datetimes.DatetimeIndex object)
        For example, a possible return could be [(2017-01-01 00:00:00, 2017-01-01 00:10:00),
                                                 (2017-01-01 00:10:00, 2017-01-01 00:20:00)]
    '''
    pattern = re.compile('^([0-9]{4})-([0-1][0-9])-([0-3][0-9])\s([0-1][0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])$')
    if pattern.match(stp) and pattern.match(etp):
        time_bounds = pd.date_range(stp, etp, freq=freq)
        sub_intervals = list(zip(time_bounds[:-1], time_bounds[1:]))
        print(len(time_bounds), len(sub_intervals))
        return sub_intervals
    else:
        raise Exception('Provided time bound is of invalid format.')


### 3. Location Preprocessing

Preprocess location related information: mapping ids and locations, then generate adjacency matrices.

In [11]:
# Load zone lookup table
zones = pd.read_csv('dataset/taxi_zone_lookup.csv')
print(f'shape is: {zones.shape}')

shape is: (265, 4)


In [12]:
yellow_zone = zones.loc[zones['Borough'] == 'Manhattan']
print(yellow_zone.shape)
img_size = yellow_zone.shape[0]

(69, 4)


### 4. Generate Tensor: 3 matrices (layers) of connection

Generate images that represent traffic states.

Define functions to generate desired outcomes:

In [16]:
def gen_snap_layers(table, bound):
    '''
    Generate Past layer, Now layer and Future layer for one snapshot.
    Params:
        table:
        bounds:
    Return:
        PNF layers, a list.
    '''
    # left bound and right bound of time interval
    assert type(bound) == tuple
    left = bound[0]
    right = bound[1]
    
    print(left, right)
    
    # no need to sort table indeed?
    projected_table = table.loc[:, ['tripid',
                                    'tpep_pickup_datetime',
                                    'tpep_dropoff_datetime',
                                    'pulocationid',
                                    'dolocationid']]

    # The condition of making snapshot should be:
    # at least one temporal end of a trip should be within the bounds:
    snap = projected_table.loc[
        ((projected_table['tpep_pickup_datetime'] >= left) &
         (projected_table['tpep_pickup_datetime'] < right)) |
        ((projected_table['tpep_dropoff_datetime'] >= left) &
         (projected_table['tpep_dropoff_datetime'] < right))]

    # temp table to generate F,P,N layers
    # keep snap intact
    temp_snap = snap.copy()

    # Use the interval to 'catch' corresponding trips.
    # future layer
    f_layer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] < right) &
                             (temp_snap['tpep_pickup_datetime'] >= left) &
                             (temp_snap['tpep_dropoff_datetime'] >= right)]
    # past layer
    p_layer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] < left) &
                             (temp_snap['tpep_dropoff_datetime'] >= left) &
                             (temp_snap['tpep_dropoff_datetime'] < right)]
    # now layer
    n_layer = temp_snap.loc[(temp_snap['tpep_pickup_datetime'] >= left) &
                             (temp_snap['tpep_dropoff_datetime'] < right)]

    # Their count should add up to total trips caught
    assert temp_snap.shape[0] == f_layer.shape[0] + p_layer.shape[0] + n_layer.shape[0]

    return p_layer, n_layer, f_layer


# Function that combines layers to an image
def gen_image(p_layer, n_layer, f_layer):
    '''
    Generate an image using given matrices.
    Params:
        p_layer: matrix of past layer
        n_layer: matrix of now layer
        f_layer: matrix of future layer
    Return:
        A PIL image.
    '''
    # create a snapshot
    snapshot = np.zeros([img_size, img_size, 3], dtype='float64')

    # unexpected zones
    left_zones = set()

    # future-Red: 0
    for _, row in f_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 0] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # past-Green: 1
    for _, row in p_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 1] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # now-Blue: 2
    for _, row in n_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 2] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # normalize
    snapshot *= 255 // snapshot.max()
    snapshot = snapshot.astype('uint8')
    image = Image.fromarray(snapshot)
    return image


# generate tensor, because saving image as intermediate data is not efficient
def gen_tensor(p_layer, n_layer, f_layer):
    '''
    Generate a tensor using given matrices.
    Params:
        p_layer: matrix of past layer
        n_layer: matrix of now layer
        f_layer: matrix of future layer
    Return:
        A torch tensor.
    '''
    # create a snapshot
    snapshot = np.zeros([img_size, img_size, 3], dtype='float64')

    # unexpected zones
    left_zones = set()

    # future-Red: 0
    for _, row in f_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 0] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # past-Green: 1
    for _, row in p_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 1] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # now-Blue: 2
    for _, row in n_layer.iterrows():
        try:
            snapshot[mp[str(row['pulocationid'])], mp[str(row['dolocationid'])], 2] += 1
        except Exception as e:
            left_zones.add(str(row['pulocationid']))
            left_zones.add(str(row['dolocationid']))

    # normalize
    snapshot *= 255 // snapshot.max()
    snapshot = torch.from_numpy(snapshot)
    return snapshot


# Function that gets a specific layer of snapshot.
def get_channel(image, layer:str):
    '''
    Get a layer of the snapshot.
    Params:
        image: PIL image
        channel: one of R-F,G-P,B-N
    Return:
        single channel image
    '''
    assert layer in ['P', 'N', 'F']
    namedict = {'P': 'G', 'N': 'B', 'F': 'R'}
    chandict = {'R':0, 'G':1, 'B':2}
    template = np.array(image)
    chan = np.zeros([*template.shape], dtype='uint8')
    chan[:,:,chandict[namedict[layer]]] = image.getchannel(namedict[layer])
    chan = Image.fromarray(chan)
    return chan

Save data path:

In [14]:
# image save directory:
visual_path = r'D:\GITHUB\GSPNet\snapshots\visualization'
data_path = r'D:\GITHUB\GSPNet\snapshots\data'

Start process:

In [15]:
start = time.time()
timelist = timesplit('2017-03-01 00:00:00', '2017-04-01 00:00:00')
# convert dtype for entire table here:
table['tpep_pickup_datetime'] = pd.to_datetime(table['tpep_pickup_datetime'])
table['tpep_dropoff_datetime'] = pd.to_datetime(table['tpep_dropoff_datetime'])
for i, bound in enumerate(timelist):
    p_layer, n_layer, f_layer = gen_snap_layers(table, bound)
    tensor = gen_tensor(p_layer, n_layer, f_layer)
    torch.save(tensor, f'{data_path}\\nytaxi_yellow_2017_mar_d{i}.pt')
    image = gen_image(p_layer, n_layer, f_layer)
    vimage = image.resize((690,690)) # multiply by factor of 100
    vimage.save(f'{visual_path}\\nytaxi_yellow_2017_mar_v{i}.jpg')
print(f'Image and tensor generation done in {time.time() - start :.2f} seconds.')

4465 4464
2017-03-01 00:00:00 2017-03-01 00:10:00


NameError: name 'sorted_table' is not defined